## SECTION 1 - Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import joblib


## SECTION 2 - Load MNIST CSV files

In [2]:
train_path = "./mnist_train.csv"
test_path  = "./mnist_test.csv"

mnist_train = pd.read_csv(train_path, header=None)
mnist_test  = pd.read_csv(test_path, header=None)

print("Train shape:", mnist_train.shape)
print("Test shape:", mnist_test.shape)


Train shape: (60000, 785)
Test shape: (10000, 785)


## SECTION 3 - Split features and labels

In [3]:

y_train_full = mnist_train.iloc[:, 0].values
X_train_full = mnist_train.iloc[:, 1:].values

y_test = mnist_test.iloc[:, 0].values
X_test = mnist_test.iloc[:, 1:].values

print("X_train_full:", X_train_full.shape)
print("y_train_full:", y_train_full.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)


X_train_full: (60000, 784)
y_train_full: (60000,)
X_test: (10000, 784)
y_test: (10000,)


## SECTION 4 - Preprocessing

In [4]:
# Normalize to 0 - 1
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, shuffle=True
)

# StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


## SECTION 5 - Train multiple models

### 5.1 - KNN

In [5]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

knn_val_preds = knn.predict(X_val_scaled)
knn_val_acc = accuracy_score(y_val, knn_val_preds)
print("KNN validation accuracy:", knn_val_acc)

for k in [3, 5, 7]:
    knn_temp = KNeighborsClassifier(n_neighbors=k)
    knn_temp.fit(X_train_scaled, y_train)
    preds_temp = knn_temp.predict(X_val_scaled)
    print(f"k = {k}, val accuracy = {accuracy_score(y_val, preds_temp)}")



KNN validation accuracy: 0.9431666666666667
k = 3, val accuracy = 0.9431666666666667
k = 5, val accuracy = 0.9431666666666667
k = 7, val accuracy = 0.9423333333333334


### 5.2 - Logistic Regression

In [6]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

log_val_preds = log_reg.predict(X_val_scaled)
log_val_acc = accuracy_score(y_val, log_val_preds)
print("Logistic Regression validation accuracy:", log_val_acc)


Logistic Regression validation accuracy: 0.9151666666666667


### 5.3 - MLP (Multi Layer Perceptron)

In [7]:
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='logistic',  
    max_iter=20,             
    verbose=False
)

mlp.fit(X_train_scaled, y_train)

mlp_val_preds = mlp.predict(X_val_scaled)
mlp_val_acc = accuracy_score(y_val, mlp_val_preds)
print("MLP validation accuracy:", mlp_val_acc)


MLP validation accuracy: 0.9653333333333334


c:\Users\pruth\Documents\Seneca College\Seneca Sem 6\CVI\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## SECTION 6 - Compare accuracies

In [8]:
results = {
    "KNN": knn_val_acc,
    "Logistic Regression": log_val_acc,
    "MLP": mlp_val_acc
}

print("\nValidation accuracies:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")



Validation accuracies:
KNN: 0.9432
Logistic Regression: 0.9152
MLP: 0.9653


## SECTION 7 - Train best model on full training set and evaluate on test set

In [9]:
best_model_name = max(results, key=results.get)
print("Best model on validation:", best_model_name)

# Refit scaler on full training data
scaler_full = StandardScaler()
X_train_full_scaled = scaler_full.fit_transform(X_train_full)
X_test_full_scaled = scaler_full.transform(X_test)

if best_model_name == "KNN":
    best_model = KNeighborsClassifier(n_neighbors=5)  
elif best_model_name == "Logistic Regression":
    best_model = LogisticRegression(max_iter=1000)
else:
    best_model = MLPClassifier(
        hidden_layer_sizes=(128, 64),
        activation='logistic',
        max_iter=30
    )

best_model.fit(X_train_full_scaled, y_train_full)

test_preds = best_model.predict(X_test_full_scaled)
test_acc = accuracy_score(y_test, test_preds)
print("Test accuracy with best model:", test_acc)


Best model on validation: MLP
Test accuracy with best model: 0.963


c:\Users\pruth\Documents\Seneca College\Seneca Sem 6\CVI\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  warnings.warn(


## SECTION 8 - Save best model

In [10]:
joblib.dump(best_model, "mnist_best_model.r")
joblib.dump(scaler_full, "mnist_scaler.r")
print("Saved best model and scaler.")


Saved best model and scaler.
